Dans ce script, nous allons tester différents modèles de type convolutionnelsur le dataset 

résultat:
* l'utilisation des batchs rend l'inférence globale plus rapide
* un modèle a besoin de beaucoup de temps / parametres pour arriver à un certain niveau d'accuracy

In [1]:
import retinoto_py as fovea
n_val_stop = 0
model_name = 'convnext_base'
args = fovea.Params(do_fovea=True, model_name=model_name, n_val_stop=n_val_stop, in_memory=False)
args

Welcome on macOS-26.1-arm64-arm-64bit-Mach-O	- Timestamp (UTC)  2025-12-05_08-07-50	 user laurentperrinet	Running on MPS device (Apple Silicon/MacOS)	 - macos_version = 26.1	 with device mps, pytorch==2.9.1
Random seed 2018 has been set.


TypeError: Params.__init__() got an unexpected keyword argument 'n_val_stop'

# testing different networks

In [2]:
args = fovea.Params()

json_filename = args.data_cache / '33_model_comparison.json'

# %rm {json_filename}  # FORCING RECOMPUTE

if json_filename.exists():
    results = fovea.pd.read_json(json_filename)
else:
    all_results = []
    for dataset in fovea.all_datasets:
        args = fovea.Params(do_fovea=True, model_name=model_name, n_val_stop=n_val_stop, in_memory=False)
        VAL_DATA_DIR = args.DATAROOT / f'Imagenet_{dataset}' / 'val'
        val_dataset = fovea.get_dataset(args, VAL_DATA_DIR, n_stop=args.n_val_stop)
        val_loader = fovea.get_loader(args, val_dataset)

        model = fovea.load_model(args)
        param_stats = fovea.count_parameters(model)
        total_layers = fovea.count_layers(model)
        tic = fovea.time.time()
        accuracy = fovea.get_validation_accuracy(args, model, val_loader, f"Model {model_name}\t dataset: {dataset}")            
        toc = fovea.time.time()
        
        this_result = { 'model_name': model_name,
                        'dataset': dataset,
                        'accuracy': accuracy,
                        'wall_clock_time': (toc-tic)/len(val_dataset),
                        'total_parameters': param_stats['total_parameters'],
                        'trainable_parameters': param_stats['trainable_parameters'],
                        'total_layers': total_layers}
        all_results.append(this_result)
    results = fovea.pd.DataFrame(all_results)
    results.to_json(json_filename, orient='records', indent=2)


In [3]:
results

,model_name,dataset,accuracy,wall_clock_time,total_parameters,trainable_parameters,total_layers
0,convnext_base,full,0.389120,0.013159,88591464,88591464,383
1,convnext_base,bbox,0.312512,0.012811,88591464,88591464,383


In [4]:
# fig, ax = fovea.plot_model_comparison(results, fovea.all_model_names, fovea.all_datasets, do_masks=[True, False], figures_folder=args.figures_folder, save_name='33_model_comparison', exts=['svg'])
